# RecallrAI Python SDK Examples

This notebook demonstrates the usage of the RecallrAI Python SDK, a system that enables AI assistants to form meaningful connections between conversations, just like human memory.

## Installation
To install the RecallrAI Python SDK, use the following command:

In [ ]:
%pip install recallrai python-dotenv --upgrade --quiet

In [ ]:
import recallrai
print(recallrai.__version__)

## Initialization

First, create a client instance with your API key and project ID:

In [ ]:
import os
from recallrai import RecallrAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Replace with your actual API key and project ID
api_key = os.getenv("RECALLRAI_API_KEY")
project_id = os.getenv("RECALLRAI_PROJECT_ID")

if not api_key or not project_id:
    raise ValueError("Please set RECALLRAI_API_KEY and RECALLRAI_PROJECT_ID in your .env file.")

client = RecallrAI(api_key=api_key, project_id=project_id)

## User Management

### Create a User

In [ ]:
from recallrai.exceptions import UserAlreadyExistsError
try:
    user = client.create_user(user_id="user123", metadata={"name": "John Doe"})
    print(f"Created user: {user.user_id}")
    print(f"User metadata: {user.metadata}")
    print(f"Created at: {user.created_at}")
except UserAlreadyExistsError as e:
    print(f"Error: {e}")

### Get a User

In [ ]:
from recallrai.exceptions import UserNotFoundError
try:
    user = client.get_user("user123")
    print(f"User metadata: {user.metadata}")
    print(f"Last active: {user.last_active_at}")
except UserNotFoundError as e:
    print(f"Error: {e}")

### List Users

In [ ]:
user_list = client.list_users(offset=0, limit=10)
print(f"Total users: {user_list.total}")
print(f"Has more users: {user_list.has_more}")

for user in user_list.users:
    print(f"User ID: {user.user_id}")
    print(f"Metadata: {user.metadata}")
    print(f"Created at: {user.created_at}")
    print(f"Last active: {user.last_active_at}")
    print("---")

### Update a User

In [ ]:
from recallrai.exceptions import UserNotFoundError, UserAlreadyExistsError
try:
    user = client.get_user("user123")
    updated_user = user.update(
        new_metadata={"name": "John Doe", "role": "admin"},
        new_user_id="john_doe"
    )
    print(f"Updated user ID: {updated_user.user_id}")
    print(f"Updated metadata: {updated_user.metadata}")
except UserNotFoundError as e:
    print(f"Error: {e}")
except UserAlreadyExistsError as e:
    print(f"Error: {e}")

### Delete a User

In [ ]:
from recallrai.exceptions import UserNotFoundError
try:
    user = client.get_user("john_doe")
    user.delete()
    print("User deleted successfully")
except UserNotFoundError as e:
    print(f"Error: {e}")

## Session Management

### Create a Session

In [ ]:
from recallrai.exceptions import UserNotFoundError
from recallrai.session import Session

try:
    # First, get the user
    user = client.get_user("user123")
    
    # Create a session for the user; auto_process_after_minutes set to -1 disables auto-processing
    session: Session = user.create_session(auto_process_after_minutes=5)
    print("Created session id:", session.session_id)
except UserNotFoundError as e:
    print(f"Error: {e}")

### Get an Existing Session

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError

try:
    # First, get the user
    user = client.get_user("user123")
    
    # Retrieve an existing session by its ID
    session = user.get_session(session_id="session-uuid")
    print("Session status:", session.get_status())
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")

### List Sessions

In [ ]:
from recallrai.exceptions import UserNotFoundError

try:
    # First, get the user
    user = client.get_user("user123")
    
    # List sessions for this user
    session_list = user.list_sessions(offset=0, limit=10)
    for session in session_list.sessions:
        print(session.session_id, session.status)
except UserNotFoundError as e:
    print(f"Error: {e}")

### Session – Adding Messages

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError, InvalidSessionStateError

try:
    # Add a user message
    session.add_user_message("Hello! How are you?")
    
    # Add an assistant message
    session.add_assistant_message("I'm an assistant. How can I help you?")
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")
except InvalidSessionStateError as e:
    print(f"Error: {e}")

### Session – Retrieving Context

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError

try:
    context = session.get_context()
    print("Memory used:", context.memory_used)
    print("Context:", context.context)
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")

### Session – Process Session

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError, InvalidSessionStateError

try:
    session.process()
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")
except InvalidSessionStateError as e:
    print(f"Error: {e}")

### Session – Get Status and Messages

In [ ]:
from recallrai.exceptions import UserNotFoundError, SessionNotFoundError
from recallrai.models import SessionStatus

try:
    status = session.get_status()
    print("Session status:", status)
    
    # Check if the session is in a specific state
    if status == SessionStatus.PROCESSED:
        print("Session has been processed")
    
    messages = session.get_messages()
    for message in messages:
        print(f"{message.role}: {message.content} at {message.timestamp}")
except UserNotFoundError as e:
    print(f"Error: {e}")
except SessionNotFoundError as e:
    print(f"Error: {e}")